In [ ]:
import pandas as pd
import pysam

In [ ]:
contig = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/consensus_annotations.csv')

In [ ]:
contig['id_num'] = contig['clonotype_id'].apply(lambda x: int(x[9:]))

In [ ]:
contig.sort_values('id_num',inplace=True)

In [ ]:
del contig['id_num']

In [ ]:
contig

In [ ]:
seq_dict =  dict()

In [ ]:
with pysam.FastxFile('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/consensus.fasta') as f:
    for read in f:
        seq_dict[read.name] = read.sequence

In [ ]:
df_seq = pd.DataFrame.from_dict(seq_dict, orient='index', columns=['seq'])

In [ ]:
df_seq

In [ ]:
df_seq = df_seq.rename_axis('consensus_id').reset_index()

In [ ]:
df_seq

In [ ]:
df_merge = pd.merge(contig,df_seq,on='consensus_id')

In [ ]:
df_merge

In [ ]:
clonotype_count = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/clonotypes.csv')

In [ ]:
clonotype_count = clonotype_count[['clonotype_id','frequency']]

In [ ]:
df_merge = pd.merge(df_merge,clonotype_count,on='clonotype_id')

In [ ]:
df_merge['id_num'] = df_merge['clonotype_id'].apply(lambda x: int(x[9:]))

df_merge.sort_values(['id_num','chain'],inplace=True)

In [ ]:
del df_merge['id_num']

In [ ]:
df_merge

In [ ]:
clonotype_id_list = list(df_merge.clonotype_id.unique())

In [ ]:
clonotype_id_list

In [ ]:
outfile = open('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220704anno_seq/anno_seq_aa.fasta','w')

In [ ]:
outfile.write("Clonotype(clonotype_id|V-gene|D-gene|J-gene|CDR3|C-gene|)\tCell_Number\tCgene\tSequence\n")
for i in clonotype_id_list:
    df_tmp = df_merge[df_merge['clonotype_id']==i]
    outfile.write(f'{list(df_tmp.clonotype_id)[0]}|')
    for i in range(df_tmp.shape[0]):
        outfile.write(f'{list(df_tmp.v_gene)[i]}|{list(df_tmp.d_gene)[i]}|{list(df_tmp.j_gene)[i]}|{list(df_tmp.cdr3)[i]}|{list(df_tmp.c_gene)[i]}|')
    outfile.write(f'\t{list(df_tmp.frequency)[0]}\t')
    outfile.write('|'.join(list(df_tmp.c_gene)))
    outfile.write('\n')
    for i in range(df_tmp.shape[0]):
        outfile.write(f'ChianType:{list(df_tmp.chain)[i]}\n{list(df_tmp.seq)[i]}\n')
        #outfile.write('\n'.join(list(df_tmp.seq)))
    # outfile.write('\n')

In [ ]:
outfile.close()

In [ ]:
# aa 
from Bio.Seq import Seq

In [ ]:
df_merge['seq_aa'] = df_merge['seq'].apply(lambda x: str(Seq(x).translate()))

In [ ]:
df_merge

In [ ]:
df_tmp = df_merge[df_merge['clonotype_id']=='clonotype2310']

In [ ]:
list(df_tmp.chain)

In [ ]:
df_tmp.sort_values('chain')

In [ ]:
# pieplot
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
contig = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/04.summarize/productive_contig_annotations.csv')

In [ ]:
contig

In [ ]:
with pysam.FastxFile('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/04.summarize/productive_contig.fasta') as f:
    for read in f:
        print(str(Seq(read.sequence).translate()))

In [ ]:
contig['seq_aa'] = contig['seq'].apply(lambda x: str(Seq(x).translate()))

In [ ]:
contig

In [ ]:
df_h = contig[contig['chain']=='IGH']
df_l = contig[contig['chain']!='IGH']

In [ ]:
df_h.sort_values('c_gene', ascending=False,inplace=True)

In [ ]:
df_h

In [ ]:
df_h.groupby('c_gene')['raw_clonotype_id'].count().to_dict()

In [ ]:
tmp_dict = df_h.groupby('c_gene')['raw_clonotype_id'].count().to_dict()

In [ ]:
del(tmp_dict['None'])

In [ ]:
tmp_dict

In [ ]:
tmp_dict,reverse_tmp_dict = sorted(tmp_dict.items(),key=lambda x:x[1]), sorted(tmp_dict.items(),key=lambda x:x[1], reverse=True)

In [ ]:
reverse_tmp_dict

In [ ]:
label, value = [], []
if len(tmp_dict) % 2 == 0:
    for i in range(len(tmp_dict)//2):
        label.append(tmp_dict[i][0])
        label.append(reverse_tmp_dict[i][0])
        value.append(tmp_dict[i][1])
        value.append(reverse_tmp_dict[i][1])
else:
    for i in range(len(tmp_dict)//2):
        label.append(tmp_dict[i][0])
        label.append(reverse_tmp_dict[i][0])
        value.append(tmp_dict[i][1])
        value.append(reverse_tmp_dict[i][1])
    label.append(tmp_dict[len(tmp_dict)//2][0])
    value.append(tmp_dict[len(tmp_dict)//2][1])

In [ ]:
label

In [ ]:
value

In [ ]:
value = list(tmp_dict.values())
label = list(tmp_dict.keys())

In [ ]:
value.sort()

In [ ]:
value

In [ ]:
label

In [ ]:
value = [value[1],value[0],value[2],value[3]]
label = [label[1],label[0],label[2],label[3]]

In [ ]:
plt.pie(value, labels=label,autopct='%.2f%%',pctdistance=0.8,explode=(0.1,0.1,0.1,0.1))
plt.title('IG types')
plt.savefig('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220713anno_seq/igl.png', dpi=280)

In [ ]:
df_h[df_h['c_gene']=='None']

In [ ]:
test = df_merge.groupby('clonotype_id')['chain'].count().to_dict()

In [ ]:
count_1=0
count_3=0
for k,v in test.items():
    if v==1:
        count_1 +=1
    if v>=3:
        count_3 +=1

In [ ]:
# 7.13 

In [ ]:
import pandas as pd
import pysam
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
airr = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/airr_rearrangement.tsv',sep='\t')

In [ ]:
airr

In [ ]:
contig = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/filtered_contig_annotations.csv')

In [ ]:
contig = contig[contig['productive']==True]

In [ ]:
contig

In [ ]:
contig = contig[['contig_id','raw_clonotype_id','cdr3']]

In [ ]:
contig.rename(columns={'contig_id':'sequence_id'},inplace=True)

In [ ]:
df_merge = pd.merge(airr,contig,on='sequence_id',how='inner')
del df_merge['clone_id']

In [ ]:
df_merge.rename(columns={'raw_clonotype_id':'clonotype_id'},inplace=True)

In [ ]:
df_merge

In [ ]:
clonotype = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/clonotypes.csv')

In [ ]:
clonotype = clonotype[['clonotype_id','frequency']]

In [ ]:
clonotype

In [ ]:
df_merge_f = pd.merge(df_merge,clonotype,on='clonotype_id')

In [ ]:
df_merge_f['chain'] = df_merge_f['v_call'].apply(lambda x:x[:3])

In [ ]:
df_merge_f

In [ ]:
df_merge_f['id_num'] = df_merge_f['clonotype_id'].apply(lambda x: int(x[9:]))
df_merge_f.sort_values(['id_num','chain'],inplace=True)

In [ ]:
del df_merge_f['id_num']
del df_merge_f['sequence']
del df_merge_f['productive']
del df_merge_f['rev_comp']
del df_merge_f['v_cigar']
del df_merge_f['d_cigar']
del df_merge_f['j_cigar']
del df_merge_f['c_cigar']
del df_merge_f['sequence_alignment']
del df_merge_f['germline_alignment']
del df_merge_f['junction']
del df_merge_f['junction_aa']
del df_merge_f['junction_length']
del df_merge_f['junction_aa_length']
del df_merge_f['v_sequence_start']
del df_merge_f['v_sequence_end']
del df_merge_f['d_sequence_start']
del df_merge_f['d_sequence_end']
del df_merge_f['j_sequence_start']
del df_merge_f['j_sequence_end']
del df_merge_f['c_sequence_start']
del df_merge_f['c_sequence_end']
del df_merge_f['consensus_count']
del df_merge_f['duplicate_count']
del df_merge_f['is_cell']

In [ ]:
df_merge_f

In [ ]:
df_merge_f.drop_duplicates(['clonotype_id','chain'],inplace=True)

In [ ]:
df_merge_f = df_merge_f.where(df_merge_f.notnull(),'None')

In [ ]:
clonotype_id_list = list(df_merge_f.clonotype_id.unique())

In [ ]:
clonotype_id_list

In [ ]:
df_merge_f

In [ ]:
outfile = open('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220713anno_seq/test.fasta','w')

In [ ]:
outfile.write("Clonotype(clonotype_id|V-gene|D-gene|J-gene|CDR3|C-gene|)\tCell_Number\tCgene\tSequence\n")
for i in clonotype_id_list:
    df_tmp = df_merge_f[df_merge_f['clonotype_id']==i]
    outfile.write(f'{list(df_tmp.clonotype_id)[0]}|')
    for i in range(df_tmp.shape[0]):
        outfile.write(f'{list(df_tmp.v_call)[i]}|{list(df_tmp.d_call)[i]}|{list(df_tmp.j_call)[i]}|{list(df_tmp.cdr3)[i]}|{list(df_tmp.c_call)[i]}|')
    outfile.write(f'\t{list(df_tmp.frequency)[0]}\t')
    outfile.write('|'.join(list(df_tmp.c_call)))
    outfile.write('\n')
    for i in range(df_tmp.shape[0]):
        outfile.write(f'ChianType:{list(df_tmp.chain)[i]}\n{list(df_tmp.sequence_aa)[i]}\n')
        #outfile.write('\n'.join(list(df_tmp.seq)))
    # outfile.write('\n')

In [ ]:
outfile.close()

In [ ]:
df_merge_f

In [ ]:
tmp_dict

In [ ]:
df_h = df_merge_f[df_merge_f['chain']=='IGH']
df_h.sort_values('c_call', ascending=False,inplace=True)

In [ ]:
tmp_dict = df_h.groupby('c_call')['clonotype_id'].count().to_dict()
value = list(tmp_dict.values())
label = list(tmp_dict.keys())

In [ ]:
value = [value[1],value[0],value[2],value[3]]
label = [label[1],label[0],label[2],label[3]]

plt.pie(value, labels=label,autopct='%.2f%%',pctdistance=0.8,explode=(0.1,0.1,0.1,0.1))
plt.title('IG types')
plt.savefig('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220704anno_seq/igl.png', dpi=280)

In [1]:
# 20220714
import pandas as pd
import pysam
import matplotlib.pyplot as plt
import pandas as pd
from Bio.Seq import Seq

In [2]:
airr = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/airr_rearrangement.tsv',sep='\t')
contig = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/filtered_contig_annotations.csv')
clonotype = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220629no_match/BCRLFBVC220616037X1CK/03.assemble/BCRLFBVC220616037X1CK/outs/clonotypes.csv')

In [3]:
contig = contig[contig['productive']==True]
contig = contig[['contig_id','raw_clonotype_id','cdr3']]
contig.rename(columns={'contig_id':'sequence_id'},inplace=True)
df_merge = pd.merge(airr,contig,on='sequence_id',how='inner')
del df_merge['clone_id']
df_merge.rename(columns={'raw_clonotype_id':'clonotype_id'},inplace=True)
clonotype = clonotype[['clonotype_id','frequency']]
df_merge_f = pd.merge(df_merge,clonotype,on='clonotype_id')
df_merge_f['chain'] = df_merge_f['v_call'].apply(lambda x:x[:3])
df_merge_f['id_num'] = df_merge_f['clonotype_id'].apply(lambda x: int(x[9:]))
df_merge_f.sort_values(['id_num','chain'],inplace=True)

In [4]:
del df_merge_f['id_num']
del df_merge_f['productive']
del df_merge_f['rev_comp']
del df_merge_f['v_cigar']
del df_merge_f['d_cigar']
del df_merge_f['j_cigar']
del df_merge_f['c_cigar']
del df_merge_f['sequence_alignment']
del df_merge_f['germline_alignment']
del df_merge_f['junction']
del df_merge_f['junction_aa']
del df_merge_f['junction_length']
del df_merge_f['junction_aa_length']
del df_merge_f['v_sequence_start']
del df_merge_f['v_sequence_end']
del df_merge_f['d_sequence_start']
del df_merge_f['d_sequence_end']
del df_merge_f['j_sequence_start']
del df_merge_f['j_sequence_end']
del df_merge_f['c_sequence_start']
del df_merge_f['c_sequence_end']
del df_merge_f['consensus_count']
del df_merge_f['duplicate_count']
del df_merge_f['is_cell']

In [5]:
df_merge_f.drop_duplicates(['clonotype_id','chain'],inplace=True)
df_merge_f = df_merge_f.where(df_merge_f.notnull(),'None')
clonotype_id_list = list(df_merge_f.clonotype_id.unique())

In [6]:
df_merge_f

,cell_id,sequence_id,sequence,sequence_aa,v_call,d_call,j_call,c_call,clonotype_id,cdr3,frequency,chain
901,AAACCTGAGTATCTCG-1,AAACCTGAGTATCTCG-1_contig_1,AATACCACCAATGCTAATACGTAGTCATCACGTTGAGACCTACATG...,MMSSAQFLGLLLLCFQGTRCDIQMTQTTSSLSASLGDRVTISCRAS...,IGKV10-96,None,IGKJ1,IGKC,clonotype1,CQQGNTLPWTF,89,IGK
2414,AAACCTGGTACTCTCC-1,AAACCTGGTACTCTCC-1_contig_1,TACCGAGCTCAGCTTCTGACTACGTTAGGCATATATGGCTGTCACC...,MAWTSLILSLLALCSGASSQAVVTQESALTTSPGGTVILTCRSSTG...,IGLV2,None,IGLJ2,IGLC2,clonotype2,CALWYSTHYVF,82,IGL
1809,AAACCTGCAGACGCTC-1,AAACCTGCAGACGCTC-1_contig_1,GGGGACCAATATTGAAAAGAATAGACCTGGTTTGTGAATTATGGCC...,MAWISLILSLLALSSGAISQAVVTQESALTTSPGETVTLTCRSSTG...,IGLV1,None,IGLJ1,IGLC1,clonotype3,CALWYSNHWVF,71,IGL
242,AAACCTGAGATGCCAG-1,AAACCTGAGATGCCAG-1_contig_1,ATCACACTGAAAACACACAGACATGAGTGTGCCCACTCAGGTCCTG...,MSVPTQVLGLLLLWLTGARCDIQMTQSPASLSASVGETVTITCRAS...,IGKV12-44,None,IGKJ2,IGKC,clonotype4,CQHHYGTPYTF,62,IGK
1278,AAACCTGCAAAGGAAG-1,AAACCTGCAAAGGAAG-1_contig_1,CCCCACCACCCACCCCAGGTTTTAATGCTACGGCTACGACCGAGTT...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...,IGKV6-15,None,IGKJ5,IGKC,clonotype5,CQQYNSYPLTF,62,IGK
...,...,...,...,...,...,...,...,...,...,...,...,...
23255,AGGGATGTCCTATTCA-1,AGGGATGTCCTATTCA-1_contig_1,TAATAGACTTGGTTTGTGAATTATGGCCTGGACTTCACTTATACTC...,MAWTSLILSLLALCSGASSQAVVTQESALTTSPGGTVILTCRSSTG...,IGLV2,None,IGLJ2,IGLC2,clonotype9620,CALWYSTHFVF,1,IGL
23257,AGGTCCGCACCTTGTC-1,AGGTCCGCACCTTGTC-1_contig_1,TGGATGAGGATGGAGATCGGAAGAGCGTCGTGTAGGAGCAAGTATG...,MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGY...,IGHV1-55,None,IGHJ3,IGHM,clonotype9621,CTRGGLRRGFAYW,1,IGH
23258,AGGTCCGCACCTTGTC-1,AGGTCCGCACCTTGTC-1_contig_2,GATCAGCATGGGCATCAAGATGGAGTCACAGACTCAGGTCTTTGTA...,MESQTQVFVYMLLWLSGVDGDIVMTQSQKFMSTSVGDRVSVTCKAS...,IGKV6-15,None,IGKJ2,IGKC,clonotype9621,CQQYNSYPYTF,1,IGK
23259,ATCACGACAAGCCGCT-1,ATCACGACAAGCCGCT-1_contig_1,GGACTCTCAGTGAGGATACACCATCAGCATGAGGGTCCTTGCTGAG...,MRVLAELLGLLLFCFLGVRCDIQMNQSPSSLSASLGDTITITCHAS...,IGKV15-103,None,IGKJ2,IGKC,clonotype9622,CQQGQSYPYTF,1,IGK


In [ ]:
xing_list = list(df_merge_f['sequence_aa'])


In [ ]:
index_list = []
for i in xing_list:
    if '*' in i:
        index_list.append(xing_list.index(i))

In [ ]:
df_merge_f_new = df_merge_f.reset_index()
del df_merge_f_new['index']

In [ ]:
df_merge_f_new

In [ ]:
df_merge_f_new

In [ ]:
str(Seq('ATGAGAGTGCTGATTCTTTTGTGGCTGTTCACAGCCTTTCCTGGTATCCTGTCTGATGTGCAGCTTCAGGAGTCGGGACCTGGCCTGGTGAAACCTTCTCAGTCTCTGTCCCTCACCTGCACTGTCACTGGCTACTCAATCACCAGTGATTATGCCTGGAACTGGATCCGGCAGTTTCCAGGAAACAAACTGGAGTGGATGGGCTACATAAGCTACAGTGGTAGCACTAGCTACAACCCATCTCTCAAAAGTCGAATCTCTATCACTCGAGACACATCCAAGAACCAGTTCTTCCTGCAGTTGAATTCTGTGACTACTGAGGACACAGCCACATATTACTGTGCAAAATATGACTACGGCTGTGCTATGGACTACTGGGGTCAAGGAACCTCAGTCACCGTCTCCTCAGGTGATAAAAAGGAACCTGACATGTTCCTCCTCTCAGAGTGCAAAGCCCGTGAGGAAAATTAATAGATATACCTGGGCTGTTTAGTAATTGGAAGTCAG').translate())

In [ ]:
outfile = open('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220714anno_seq/anno_seq_aa.fasta','w')

In [ ]:
outfile.write("Clonotype(clonotype_id|V-gene|D-gene|J-gene|CDR3|C-gene|)\tCell_Number\tCgene\tSequence\n")
for i in clonotype_id_list:
    df_tmp = df_merge_f[df_merge_f['clonotype_id']==i]
    outfile.write(f'{list(df_tmp.clonotype_id)[0]}|')
    for i in range(df_tmp.shape[0]):
        outfile.write(f'{list(df_tmp.v_call)[i]}|{list(df_tmp.d_call)[i]}|{list(df_tmp.j_call)[i]}|{list(df_tmp.cdr3)[i]}|{list(df_tmp.c_call)[i]}|')
    outfile.write(f'\t{list(df_tmp.frequency)[0]}\t')
    outfile.write('|'.join(list(df_tmp.c_call)))
    outfile.write('\n')
    for i in range(df_tmp.shape[0]):
        outfile.write(f'ChianType:{list(df_tmp.chain)[i]}\n{list(df_tmp.sequence_aa)[i]}\n')
        #outfile.write('\n'.join(list(df_tmp.seq)))
    # outfile.write('\n')

In [ ]:
outfile.close()

In [ ]:
outfile = open('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220714anno_seq/anno_seq_nt.fasta','w')

In [ ]:
outfile.write("Clonotype(clonotype_id|V-gene|D-gene|J-gene|CDR3|C-gene|)\tCell_Number\tCgene\tSequence\n")
for i in clonotype_id_list:
    df_tmp = df_merge_f[df_merge_f['clonotype_id']==i]
    outfile.write(f'{list(df_tmp.clonotype_id)[0]}|')
    for i in range(df_tmp.shape[0]):
        outfile.write(f'{list(df_tmp.v_call)[i]}|{list(df_tmp.d_call)[i]}|{list(df_tmp.j_call)[i]}|{list(df_tmp.cdr3)[i]}|{list(df_tmp.c_call)[i]}|')
    outfile.write(f'\t{list(df_tmp.frequency)[0]}\t')
    outfile.write('|'.join(list(df_tmp.c_call)))
    outfile.write('\n')
    for i in range(df_tmp.shape[0]):
        seq_nt = list(df_tmp.sequence)[i]
        seq_aa = list(df_tmp.sequence_aa)[i]
        for j in range(len(seq_nt)):
            if str(Seq(seq_nt[j:]).translate()) == seq_aa:
                seq_nt_write = seq_nt[j:]
                if len(seq_nt_write) % 3 == 1:
                    seq_nt_write = seq_nt[j:-1]
                elif len(seq_nt_write) % 3 == 2:
                    seq_nt_write = seq_nt[j:-2]
                outfile.write(f'ChianType:{list(df_tmp.chain)[i]}\n{seq_nt_write}\n')
                break

In [ ]:
outfile.close()

In [ ]:
from Bio.Seq import Seq

In [ ]:
with open('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220714anno_seq/anno_seq_nt.fasta') as f:
        lines_nt = f.readlines()
        lines_nt = [line.strip('\n') for line in lines_nt]
with open('/SGRNJ06/randd/USER/cjj/celedev/vdj10x/20220714anno_seq/anno_seq_aa.fasta') as f:
        lines_aa = f.readlines()
        lines_aa = [line.strip('\n') for line in lines_aa]

In [ ]:
for i in range(3,150,3):
    print(len(lines_nt[i]))

In [ ]:
a = 'ATGGGATGGAGCTGTATCATCCTCTTCTTGGTAGCAACAGCTACAGGTGTCCACTCCCAGGTCCAACTGCAGCAACCTGGGTCTGAGCTGGTGAGGCCTGGAGCTTCAGTGAAGCTGTCCTGCAAGGCTTCTGGCTACACATTCACCAGCTACTGGATGCACTGGGTGAAGCAGAGGCATGGACAAGGCCTTGAGTGGATTGGAAATATTTATCCTGGTAGTGGTAGTACTAACTACGATGAGAAGTTCAAGAGCAAGGGCACACTGACTGTAGACACATCCTCCAGCACAGCCTACATGCACCTCAGCAGCCTGACATCTGAGGACTCTGCGGTCTATTACTGTACAAGGGGAGGATTACGGAGGGGGTTTGCTTACTGGGGCCAAGGGACTCTGGTCACTGTCTCTGCAGAGAGTCAGTCCTTCCCAAATGTC'

In [ ]:
len(a)

In [ ]:
str(Seq(a).translate()) == 'MGWSCIILFLVATATGVHSQVQLQQPGSELVRPGASVKLSCKASGYTFTSYWMHWVKQRHGQGLEWIGNIYPGSGSTNYDEKFKSKGTLTVDTSSSTAYMHLSSLTSEDSAVYYCTRGGLRRGFAYWGQGTLVTVSAESQSFPNV'

In [ ]:
for line in lines_aa:
    if len(line)>=200:
        if len(line) % 3 != 0:
            print(line)

In [ ]:
for line in lines_aa:
    if '*' in line:
        print(line)

In [ ]:
df_aa = list(df_merge_f.sequence_aa)
df_nt = list(df_merge_f.sequence)

In [ ]:
count = 0
for i in df_list:
    if '*' in i:
        print(df_list.index(i))
        count+=1
        

In [ ]:
df_aa[512]

In [ ]:
df_nt[512]

In [ ]:
for i in range(len(df_nt[512])):
    if str(Seq(df_nt[512][i:]).translate()) == 'MVFTPQILGLMLFWISASRGDIVLTQSPATLSVTPGDSVSLSCRASQSISNNLHWYQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPLTFGAGTKLELKLSS*HLEVPQ':
        print(df_nt[512][i:])

In [ ]:
count